Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Matthew Sah"
COLLABORATORS = ""

---

# Introduction 
This is the first of four labs that you will do in this course.  In this lab, you will parse a variant call format file (https://en.wikipedia.org/wiki/Variant_Call_Format). This file has one thousand variants in it. Each line after the header lines is a variant. Open the variant file and inspect its contents. You will parse each line and transform it into a JSON. The output will look something like this (I have shortened the output):
```
{
        "ALT": "G",
        "CHROM": "4",
        "FILTER": "PASS",
        "ID": ".",
        "INFO": {
           
            "Gene.ensGene": "ENSG00000109471,ENSG00000138684",
            "Gene.refGene": "IL2,IL21",
            "GeneDetail.ensGene": "dist=38306,dist=117597",
            "GeneDetail.refGene": "dist=38536,dist=117597"
        },
        "POS": 123416186,
        "QUAL" :23.25,
        "REF": "A",
        "SAMPLE": {
            "XG102": {
                "AD": "51,8",
                "DP": "59",
                "GQ": "32",
                "GT": "0/1",
                "PL": "32,0,1388"
            }
    }
```
The lab will guide you through writing small functions that will be used to generate the JSON above. Note that the method outlined in this lab is not necessarily optimal. It is just one way of parsing the file and teaching you how to break a large problem into small parts. 

Note that the header which corresponds to the columns starts with one hash/pound (#) symbol. Skip all headers that start with two hashes/pounds (##).

# Part 1 (3 points)
Write a function whose input is a string. This function determines the data type of the input string. The data types can be a float, int, or string. 

In [2]:
def determine_data_type(value):
    
    try :
        temp_value = float(value) 
        try:
            temp_value = int(value) 
            return int
        except:  
            return float
    except:
        return str
    
    raise NotImplementedError()
    
determine_data_type('4') == int

True

In [3]:
assert determine_data_type('1.2') == float
assert determine_data_type('4') == int
assert determine_data_type('EAS503') == str

# Part 2 (4 points)
Write a function whose input is a list of strings.  This function determines the correct data type of all the elements in the list. For example, ['1', '2', '3'] is int, ['1.1', '2.2', '3.3'] is float, ['1.1', '2', '3.3'] is also float, and ['1.1', '234String', '3.3'] is str. The purpose of this function to figure out what to cast an array of strings to. Some lists might be all ints, in which case the data type is int. Some might be a mixure of ints and floats, in which case the data type will be float. Some lists might be a mixture of ints, floats, and strings, in which case the data type of the list will be string. 

In [4]:
def determine_data_type_of_list(values): 
    #print(values)
    #print(len(values))
    #print(type(values))
    if type(values) == list:
        do_float = 0
        do_string = 0
        do_int = 0
        for x in values: 
            #print(x)
            try: 
                test_x_float = float(x) 
                try:
                    test_x_int = int(x) 
                except:
                    test_x_int = float(x)
                    do_float += 1
                #print(test_x_int, test_x_float)
                if int(test_x_float) == test_x_int:  
                    do_int +=1
                else:
                    do_float +=1 
            except:              
                do_string +=1
                
        #print(do_string, do_float, do_int)
        if do_string >0:
            #print("return str")
            return str
        elif do_float > 0 :
            #print("return float")
            return float
        else :
            #print("return int")
            return int 
    else:
        return determine_data_type(values)


In [5]:
assert determine_data_type_of_list(['1', '2', '3']) == int
assert determine_data_type_of_list(['1.1', '2.2', '3.3']) == float
assert determine_data_type_of_list(['1.1', '2', '3.3']) == float
assert determine_data_type_of_list(['1.1', '234String', '3.3']) == str


# Part 3 (3 points)
Write a function whose inputs a format field and a sample field. The format field looks like `format_field = 'GT:AD:DP:GQ:PGT:PID:PL'` and
the sample field looks like

```
sample_field = {'XG102': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG103': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG104': '0/1:34,38:72:99:.:.:938,0,796',
             'XG202': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG203': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG204': '0/1:34,38:72:99:.:.:938,0,796',
             'XG302': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG303': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG304': '0/1:34,38:72:99:.:.:938,0,796',
             'XG402': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG403': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG404': '0/1:34,38:72:99:.:.:938,0,796'}
```
Transform the inputs such that the output looks like this:
```
output = {'XG102': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG103': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG104': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG202': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG203': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG204': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG302': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG303': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG304': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG402': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG403': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG404': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'}}
```


In [6]:
format_field = "GT:AD:DP:GQ:PGT:PID:PL"
sample_field = {'XG102': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG103': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG104': '0/1:34,38:72:99:.:.:938,0,796',
             'XG202': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG203': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG204': '0/1:34,38:72:99:.:.:938,0,796',
             'XG302': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG303': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG304': '0/1:34,38:72:99:.:.:938,0,796',
             'XG402': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG403': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG404': '0/1:34,38:72:99:.:.:938,0,796'}
#import json

def format_sample_fields(format_field, sample_field):
    # Seperate data by :
    output = {}
    for x in sample_field:
        
        to_seperate_value_field = sample_field[x]
        
        seperated_value_field = to_seperate_value_field.split(":")
        #print(seperated_value_field)
        
        to_seperate_format_field =  format_field.split(":")
        #print(to_seperate_format_field)
        
        temp_sorted_value_list = {}
        for i in range( 0, len(to_seperate_format_field)):
            temp_sorted_value_list[to_seperate_format_field[i]] = seperated_value_field[i]
        #print(temp_sorted_value_list)
        
        output[x] = temp_sorted_value_list
    #print(output)
    return output

#tester = format_sample_fields(format_field, sample_field)
#print(json.dumps(tester))
#print("")
#print(tester['XG102'] )

In [7]:
solution = {'XG102': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG103': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG104': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG202': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG203': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG204': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG302': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG303': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG304': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'},
 'XG402': {'AD': '0,76',
           'DP': '76',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '1|1',
           'PID': '48306945_C_G',
           'PL': '3353,229,0'},
 'XG403': {'AD': '0,52',
           'DP': '52',
           'GQ': '99',
           'GT': '1/1',
           'PGT': '.',
           'PID': '.',
           'PL': '1517,156,0'},
 'XG404': {'AD': '34,38',
           'DP': '72',
           'GQ': '99',
           'GT': '0/1',
           'PGT': '.',
           'PID': '.',
           'PL': '938,0,796'}}

format_field = "GT:AD:DP:GQ:PGT:PID:PL"
sample_field = {'XG102': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG103': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG104': '0/1:34,38:72:99:.:.:938,0,796',
             'XG202': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG203': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG204': '0/1:34,38:72:99:.:.:938,0,796',
             'XG302': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG303': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG304': '0/1:34,38:72:99:.:.:938,0,796',
             'XG402': '1/1:0,76:76:99:1|1:48306945_C_G:3353,229,0',
             'XG403': '1/1:0,52:52:99:.:.:1517,156,0',
             'XG404': '0/1:34,38:72:99:.:.:938,0,796'}

format_output = format_sample_fields(format_field, sample_field)

assert format_output['XG102'] == solution ['XG102']
assert format_output['XG302'] == solution ['XG302']
assert format_output['XG404'] == solution ['XG404']

# Part 4 (10 points) 
Write a function whose inputs are a list containing the vcf header and a variant line.  The function should return a dictionary using the header as keys and the variant line as values. The function should use the `format_sample_fields` you wrote previously to format the sample fields. The output of the first line looks like this:
```
{'ALT': 'G',
 'CHROM': '4',
 'FILTER': 'PASS',
 'ID': '.',
 'INFO': 'AC=1;AF=0.167;AN=6;BaseQRankSum=-2.542;ClippingRankSum=0;DP=180;ExcessHet=3.0103;FS=0;MLEAC=1;MLEAF=0.167;MQ=52.77;MQRankSum=-4.631;QD=0.39;ReadPosRankSum=1.45;SOR=0.758;VQSLOD=-8.209;culprit=MQ;ANNOVAR_DATE=2018-04-16;Func.refGene=intergenic;Gene.refGene=IL2\\x3bIL21;GeneDetail.refGene=dist\\x3d38536\\x3bdist\\x3d117597;ExonicFunc.refGene=.;AAChange.refGene=.;Func.ensGene=intergenic;Gene.ensGene=ENSG00000109471\\x3bENSG00000138684;GeneDetail.ensGene=dist\\x3d38306\\x3bdist\\x3d117597;ExonicFunc.ensGene=.;AAChange.ensGene=.;cytoBand=4q27;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=.;SIFT_converted_rankscore=.;SIFT_pred=.;Polyphen2_HDIV_score=.;Polyphen2_HDIV_rankscore=.;Polyphen2_HDIV_pred=.;Polyphen2_HVAR_score=.;Polyphen2_HVAR_rankscore=.;Polyphen2_HVAR_pred=.;LRT_score=.;LRT_converted_rankscore=.;LRT_pred=.;MutationTaster_score=.;MutationTaster_converted_rankscore=.;MutationTaster_pred=.;MutationAssessor_score=.;MutationAssessor_score_rankscore=.;MutationAssessor_pred=.;FATHMM_score=.;FATHMM_converted_rankscore=.;FATHMM_pred=.;PROVEAN_score=.;PROVEAN_converted_rankscore=.;PROVEAN_pred=.;VEST3_score=.;VEST3_rankscore=.;MetaSVM_score=.;MetaSVM_rankscore=.;MetaSVM_pred=.;MetaLR_score=.;MetaLR_rankscore=.;MetaLR_pred=.;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=.;CADD_raw_rankscore=.;CADD_phred=.;DANN_score=.;DANN_rankscore=.;fathmm-MKL_coding_score=.;fathmm-MKL_coding_rankscore=.;fathmm-MKL_coding_pred=.;Eigen_coding_or_noncoding=.;Eigen-raw=.;Eigen-PC-raw=.;GenoCanyon_score=.;GenoCanyon_score_rankscore=.;integrated_fitCons_score=.;integrated_fitCons_score_rankscore=.;integrated_confidence_value=.;GERP++_RS=.;GERP++_RS_rankscore=.;phyloP100way_vertebrate=.;phyloP100way_vertebrate_rankscore=.;phyloP20way_mammalian=.;phyloP20way_mammalian_rankscore=.;phastCons100way_vertebrate=.;phastCons100way_vertebrate_rankscore=.;phastCons20way_mammalian=.;phastCons20way_mammalian_rankscore=.;SiPhy_29way_logOdds=.;SiPhy_29way_logOdds_rankscore=.;Interpro_domain=.;GTEx_V6_gene=.;GTEx_V6_tissue=.;esp6500siv2_all=.;esp6500siv2_aa=.;esp6500siv2_ea=.;ExAC_ALL=.;ExAC_AFR=.;ExAC_AMR=.;ExAC_EAS=.;ExAC_FIN=.;ExAC_NFE=.;ExAC_OTH=.;ExAC_SAS=.;ExAC_nontcga_ALL=.;ExAC_nontcga_AFR=.;ExAC_nontcga_AMR=.;ExAC_nontcga_EAS=.;ExAC_nontcga_FIN=.;ExAC_nontcga_NFE=.;ExAC_nontcga_OTH=.;ExAC_nontcga_SAS=.;ExAC_nonpsych_ALL=.;ExAC_nonpsych_AFR=.;ExAC_nonpsych_AMR=.;ExAC_nonpsych_EAS=.;ExAC_nonpsych_FIN=.;ExAC_nonpsych_NFE=.;ExAC_nonpsych_OTH=.;ExAC_nonpsych_SAS=.;1000g2015aug_all=.;1000g2015aug_afr=.;1000g2015aug_amr=.;1000g2015aug_eur=.;1000g2015aug_sas=.;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=.;avsnp150=.;CADD13_RawScore=0.015973;CADD13_PHRED=2.741;Eigen=-0.3239;REVEL=.;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0003;gnomAD_genome_AFR=0.0001;gnomAD_genome_AMR=0;gnomAD_genome_ASJ=0;gnomAD_genome_EAS=0.0007;gnomAD_genome_FIN=0.0009;gnomAD_genome_NFE=0.0002;gnomAD_genome_OTH=0.0011;gerp++gt2=.;cosmic70=.;InterVar_automated=.;PVS1=.;PS1=.;PS2=.;PS3=.;PS4=.;PM1=.;PM2=.;PM3=.;PM4=.;PM5=.;PM6=.;PP1=.;PP2=.;PP3=.;PP4=.;PP5=.;BA1=.;BS1=.;BS2=.;BS3=.;BS4=.;BP1=.;BP2=.;BP3=.;BP4=.;BP5=.;BP6=.;BP7=.;Kaviar_AF=.;Kaviar_AC=.;Kaviar_AN=.;ALLELE_END',
 'POS': '123416186',
 'QUAL': '23.25',
 'REF': 'A',
 'SAMPLE': {'XG102': {'AD': '51,8',
            'DP': '59',
            'GQ': '32',
            'GT': '0/1',
            'PL': '32,0,1388'},
       'XG103': {'AD': '47,0',
            'DP': '47',
            'GQ': '99',
            'GT': '0/0',
            'PL': '0,114,1353'},
       'XG104': {'AD': '74,0',
            'DP': '74',
            'GQ': '51',
            'GT': '0/0',
            'PL': '0,51,1827'},
       'XG202': {'AD': '51,8',
            'DP': '59',
            'GQ': '32',
            'GT': '0/1',
            'PL': '32,0,1388'},
       'XG203': {'AD': '47,0',
            'DP': '47',
            'GQ': '99',
            'GT': '0/0',
            'PL': '0,114,1353'},
       'XG204': {'AD': '74,0',
            'DP': '74',
            'GQ': '51',
            'GT': '0/0',
            'PL': '0,51,1827'},
       'XG302': {'AD': '51,8',
            'DP': '59',
            'GQ': '32',
            'GT': '0/1',
            'PL': '32,0,1388'},
       'XG303': {'AD': '47,0',
            'DP': '47',
            'GQ': '99',
            'GT': '0/0',
            'PL': '0,114,1353'},
       'XG304': {'AD': '74,0',
            'DP': '74',
            'GQ': '51',
            'GT': '0/0',
            'PL': '0,51,1827'},
       'XG402': {'AD': '51,8',
            'DP': '59',
            'GQ': '32',
            'GT': '0/1',
            'PL': '32,0,1388'},
       'XG403': {'AD': '47,0',
            'DP': '47',
            'GQ': '99',
            'GT': '0/0',
            'PL': '0,114,1353'},
       'XG404': {'AD': '74,0',
            'DP': '74',
            'GQ': '51',
            'GT': '0/0',
            'PL': '0,51,1827'}}}
            
```



In [8]:
header = "CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	XG102	XG103	XG104	XG202	XG203	XG204	XG302	XG303	XG304	XG402	XG403	XG404".split('\t')
line = """4	123416186	.	A	G	23.25	PASS	AC=1;AF=0.167;AN=6;BaseQRankSum=-2.542;ClippingRankSum=0;DP=180;ExcessHet=3.0103;FS=0;MLEAC=1;MLEAF=0.167;MQ=52.77;MQRankSum=-4.631;QD=0.39;ReadPosRankSum=1.45;SOR=0.758;VQSLOD=-8.209;culprit=MQ;ANNOVAR_DATE=2018-04-16;Func.refGene=intergenic;Gene.refGene=IL2\x3bIL21;GeneDetail.refGene=dist\x3d38536\x3bdist\x3d117597;ExonicFunc.refGene=.;AAChange.refGene=.;Func.ensGene=intergenic;Gene.ensGene=ENSG00000109471\x3bENSG00000138684;GeneDetail.ensGene=dist\x3d38306\x3bdist\x3d117597;ExonicFunc.ensGene=.;AAChange.ensGene=.;cytoBand=4q27;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=.;SIFT_converted_rankscore=.;SIFT_pred=.;Polyphen2_HDIV_score=.;Polyphen2_HDIV_rankscore=.;Polyphen2_HDIV_pred=.;Polyphen2_HVAR_score=.;Polyphen2_HVAR_rankscore=.;Polyphen2_HVAR_pred=.;LRT_score=.;LRT_converted_rankscore=.;LRT_pred=.;MutationTaster_score=.;MutationTaster_converted_rankscore=.;MutationTaster_pred=.;MutationAssessor_score=.;MutationAssessor_score_rankscore=.;MutationAssessor_pred=.;FATHMM_score=.;FATHMM_converted_rankscore=.;FATHMM_pred=.;PROVEAN_score=.;PROVEAN_converted_rankscore=.;PROVEAN_pred=.;VEST3_score=.;VEST3_rankscore=.;MetaSVM_score=.;MetaSVM_rankscore=.;MetaSVM_pred=.;MetaLR_score=.;MetaLR_rankscore=.;MetaLR_pred=.;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=.;CADD_raw_rankscore=.;CADD_phred=.;DANN_score=.;DANN_rankscore=.;fathmm-MKL_coding_score=.;fathmm-MKL_coding_rankscore=.;fathmm-MKL_coding_pred=.;Eigen_coding_or_noncoding=.;Eigen-raw=.;Eigen-PC-raw=.;GenoCanyon_score=.;GenoCanyon_score_rankscore=.;integrated_fitCons_score=.;integrated_fitCons_score_rankscore=.;integrated_confidence_value=.;GERP++_RS=.;GERP++_RS_rankscore=.;phyloP100way_vertebrate=.;phyloP100way_vertebrate_rankscore=.;phyloP20way_mammalian=.;phyloP20way_mammalian_rankscore=.;phastCons100way_vertebrate=.;phastCons100way_vertebrate_rankscore=.;phastCons20way_mammalian=.;phastCons20way_mammalian_rankscore=.;SiPhy_29way_logOdds=.;SiPhy_29way_logOdds_rankscore=.;Interpro_domain=.;GTEx_V6_gene=.;GTEx_V6_tissue=.;esp6500siv2_all=.;esp6500siv2_aa=.;esp6500siv2_ea=.;ExAC_ALL=.;ExAC_AFR=.;ExAC_AMR=.;ExAC_EAS=.;ExAC_FIN=.;ExAC_NFE=.;ExAC_OTH=.;ExAC_SAS=.;ExAC_nontcga_ALL=.;ExAC_nontcga_AFR=.;ExAC_nontcga_AMR=.;ExAC_nontcga_EAS=.;ExAC_nontcga_FIN=.;ExAC_nontcga_NFE=.;ExAC_nontcga_OTH=.;ExAC_nontcga_SAS=.;ExAC_nonpsych_ALL=.;ExAC_nonpsych_AFR=.;ExAC_nonpsych_AMR=.;ExAC_nonpsych_EAS=.;ExAC_nonpsych_FIN=.;ExAC_nonpsych_NFE=.;ExAC_nonpsych_OTH=.;ExAC_nonpsych_SAS=.;1000g2015aug_all=.;1000g2015aug_afr=.;1000g2015aug_amr=.;1000g2015aug_eur=.;1000g2015aug_sas=.;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=.;avsnp150=.;CADD13_RawScore=0.015973;CADD13_PHRED=2.741;Eigen=-0.3239;REVEL=.;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0003;gnomAD_genome_AFR=0.0001;gnomAD_genome_AMR=0;gnomAD_genome_ASJ=0;gnomAD_genome_EAS=0.0007;gnomAD_genome_FIN=0.0009;gnomAD_genome_NFE=0.0002;gnomAD_genome_OTH=0.0011;gerp++gt2=.;cosmic70=.;InterVar_automated=.;PVS1=.;PS1=.;PS2=.;PS3=.;PS4=.;PM1=.;PM2=.;PM3=.;PM4=.;PM5=.;PM6=.;PP1=.;PP2=.;PP3=.;PP4=.;PP5=.;BA1=.;BS1=.;BS2=.;BS3=.;BS4=.;BP1=.;BP2=.;BP3=.;BP4=.;BP5=.;BP6=.;BP7=.;Kaviar_AF=.;Kaviar_AC=.;Kaviar_AN=.;ALLELE_END	GT:AD:DP:GQ:PL	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827"""
#print(header)
#print(line)
def create_dict_from_line(header, line):
    
    return_dict = {}
    new_line = line.split()
    new_header = header
    split_format = ""
    split_combo = {}
    met_format = False
    
    for i in range(0, len(new_header)):
        if new_header[i] != "FORMAT" and met_format == False:
            return_dict[new_header[i]] = new_line[i]
        elif new_header[i] != "FORMAT" and  met_format == True:
            split_combo[new_header[i]] = new_line[i]
        else: 
            split_format = new_line[i] 
            met_format = True
    
    # use function for Q3
    format_output = format_sample_fields(split_format, split_combo) 
    return_dict["SAMPLE"] = format_output
    sorted_return_dict = {}

    # format answer
    for i in sorted(return_dict):
        sorted_return_dict[i] = return_dict[i]
    #print(sorted_return_dict)
    return sorted_return_dict


In [9]:
solution = {'INFO': 'AC=1;AF=0.167;AN=6;BaseQRankSum=-2.542;ClippingRankSum=0;DP=180;ExcessHet=3.0103;FS=0;MLEAC=1;MLEAF=0.167;MQ=52.77;MQRankSum=-4.631;QD=0.39;ReadPosRankSum=1.45;SOR=0.758;VQSLOD=-8.209;culprit=MQ;ANNOVAR_DATE=2018-04-16;Func.refGene=intergenic;Gene.refGene=IL2;IL21;GeneDetail.refGene=dist=38536;dist=117597;ExonicFunc.refGene=.;AAChange.refGene=.;Func.ensGene=intergenic;Gene.ensGene=ENSG00000109471;ENSG00000138684;GeneDetail.ensGene=dist=38306;dist=117597;ExonicFunc.ensGene=.;AAChange.ensGene=.;cytoBand=4q27;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=.;SIFT_converted_rankscore=.;SIFT_pred=.;Polyphen2_HDIV_score=.;Polyphen2_HDIV_rankscore=.;Polyphen2_HDIV_pred=.;Polyphen2_HVAR_score=.;Polyphen2_HVAR_rankscore=.;Polyphen2_HVAR_pred=.;LRT_score=.;LRT_converted_rankscore=.;LRT_pred=.;MutationTaster_score=.;MutationTaster_converted_rankscore=.;MutationTaster_pred=.;MutationAssessor_score=.;MutationAssessor_score_rankscore=.;MutationAssessor_pred=.;FATHMM_score=.;FATHMM_converted_rankscore=.;FATHMM_pred=.;PROVEAN_score=.;PROVEAN_converted_rankscore=.;PROVEAN_pred=.;VEST3_score=.;VEST3_rankscore=.;MetaSVM_score=.;MetaSVM_rankscore=.;MetaSVM_pred=.;MetaLR_score=.;MetaLR_rankscore=.;MetaLR_pred=.;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=.;CADD_raw_rankscore=.;CADD_phred=.;DANN_score=.;DANN_rankscore=.;fathmm-MKL_coding_score=.;fathmm-MKL_coding_rankscore=.;fathmm-MKL_coding_pred=.;Eigen_coding_or_noncoding=.;Eigen-raw=.;Eigen-PC-raw=.;GenoCanyon_score=.;GenoCanyon_score_rankscore=.;integrated_fitCons_score=.;integrated_fitCons_score_rankscore=.;integrated_confidence_value=.;GERP++_RS=.;GERP++_RS_rankscore=.;phyloP100way_vertebrate=.;phyloP100way_vertebrate_rankscore=.;phyloP20way_mammalian=.;phyloP20way_mammalian_rankscore=.;phastCons100way_vertebrate=.;phastCons100way_vertebrate_rankscore=.;phastCons20way_mammalian=.;phastCons20way_mammalian_rankscore=.;SiPhy_29way_logOdds=.;SiPhy_29way_logOdds_rankscore=.;Interpro_domain=.;GTEx_V6_gene=.;GTEx_V6_tissue=.;esp6500siv2_all=.;esp6500siv2_aa=.;esp6500siv2_ea=.;ExAC_ALL=.;ExAC_AFR=.;ExAC_AMR=.;ExAC_EAS=.;ExAC_FIN=.;ExAC_NFE=.;ExAC_OTH=.;ExAC_SAS=.;ExAC_nontcga_ALL=.;ExAC_nontcga_AFR=.;ExAC_nontcga_AMR=.;ExAC_nontcga_EAS=.;ExAC_nontcga_FIN=.;ExAC_nontcga_NFE=.;ExAC_nontcga_OTH=.;ExAC_nontcga_SAS=.;ExAC_nonpsych_ALL=.;ExAC_nonpsych_AFR=.;ExAC_nonpsych_AMR=.;ExAC_nonpsych_EAS=.;ExAC_nonpsych_FIN=.;ExAC_nonpsych_NFE=.;ExAC_nonpsych_OTH=.;ExAC_nonpsych_SAS=.;1000g2015aug_all=.;1000g2015aug_afr=.;1000g2015aug_amr=.;1000g2015aug_eur=.;1000g2015aug_sas=.;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=.;avsnp150=.;CADD13_RawScore=0.015973;CADD13_PHRED=2.741;Eigen=-0.3239;REVEL=.;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0003;gnomAD_genome_AFR=0.0001;gnomAD_genome_AMR=0;gnomAD_genome_ASJ=0;gnomAD_genome_EAS=0.0007;gnomAD_genome_FIN=0.0009;gnomAD_genome_NFE=0.0002;gnomAD_genome_OTH=0.0011;gerp++gt2=.;cosmic70=.;InterVar_automated=.;PVS1=.;PS1=.;PS2=.;PS3=.;PS4=.;PM1=.;PM2=.;PM3=.;PM4=.;PM5=.;PM6=.;PP1=.;PP2=.;PP3=.;PP4=.;PP5=.;BA1=.;BS1=.;BS2=.;BS3=.;BS4=.;BP1=.;BP2=.;BP3=.;BP4=.;BP5=.;BP6=.;BP7=.;Kaviar_AF=.;Kaviar_AC=.;Kaviar_AN=.;ALLELE_END', 'SAMPLE': {'XG404': {'GT': '0/0', 'GQ': '51', 'AD': '74,0', 'DP': '74', 'PL': '0,51,1827'}, 'XG402': {'GT': '0/1', 'GQ': '32', 'AD': '51,8', 'DP': '59', 'PL': '32,0,1388'}, 'XG403': {'GT': '0/0', 'GQ': '99', 'AD': '47,0', 'DP': '47', 'PL': '0,114,1353'}, 'XG303': {'GT': '0/0', 'GQ': '99', 'AD': '47,0', 'DP': '47', 'PL': '0,114,1353'}, 'XG302': {'GT': '0/1', 'GQ': '32', 'AD': '51,8', 'DP': '59', 'PL': '32,0,1388'}, 'XG304': {'GT': '0/0', 'GQ': '51', 'AD': '74,0', 'DP': '74', 'PL': '0,51,1827'}, 'XG202': {'GT': '0/1', 'GQ': '32', 'AD': '51,8', 'DP': '59', 'PL': '32,0,1388'}, 'XG203': {'GT': '0/0', 'GQ': '99', 'AD': '47,0', 'DP': '47', 'PL': '0,114,1353'}, 'XG204': {'GT': '0/0', 'GQ': '51', 'AD': '74,0', 'DP': '74', 'PL': '0,51,1827'}, 'XG103': {'GT': '0/0', 'GQ': '99', 'AD': '47,0', 'DP': '47', 'PL': '0,114,1353'}, 'XG102': {'GT': '0/1', 'GQ': '32', 'AD': '51,8', 'DP': '59', 'PL': '32,0,1388'}, 'XG104': {'GT': '0/0', 'GQ': '51', 'AD': '74,0', 'DP': '74', 'PL': '0,51,1827'}}, 'CHROM': '4', 'POS': '123416186', 'FILTER': 'PASS', 'QUAL': '23.25', 'ALT': 'G', 'REF': 'A', 'ID': '.'}
header = "CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	XG102	XG103	XG104	XG202	XG203	XG204	XG302	XG303	XG304	XG402	XG403	XG404".split('\t')
line = """4	123416186	.	A	G	23.25	PASS	AC=1;AF=0.167;AN=6;BaseQRankSum=-2.542;ClippingRankSum=0;DP=180;ExcessHet=3.0103;FS=0;MLEAC=1;MLEAF=0.167;MQ=52.77;MQRankSum=-4.631;QD=0.39;ReadPosRankSum=1.45;SOR=0.758;VQSLOD=-8.209;culprit=MQ;ANNOVAR_DATE=2018-04-16;Func.refGene=intergenic;Gene.refGene=IL2\x3bIL21;GeneDetail.refGene=dist\x3d38536\x3bdist\x3d117597;ExonicFunc.refGene=.;AAChange.refGene=.;Func.ensGene=intergenic;Gene.ensGene=ENSG00000109471\x3bENSG00000138684;GeneDetail.ensGene=dist\x3d38306\x3bdist\x3d117597;ExonicFunc.ensGene=.;AAChange.ensGene=.;cytoBand=4q27;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=.;SIFT_converted_rankscore=.;SIFT_pred=.;Polyphen2_HDIV_score=.;Polyphen2_HDIV_rankscore=.;Polyphen2_HDIV_pred=.;Polyphen2_HVAR_score=.;Polyphen2_HVAR_rankscore=.;Polyphen2_HVAR_pred=.;LRT_score=.;LRT_converted_rankscore=.;LRT_pred=.;MutationTaster_score=.;MutationTaster_converted_rankscore=.;MutationTaster_pred=.;MutationAssessor_score=.;MutationAssessor_score_rankscore=.;MutationAssessor_pred=.;FATHMM_score=.;FATHMM_converted_rankscore=.;FATHMM_pred=.;PROVEAN_score=.;PROVEAN_converted_rankscore=.;PROVEAN_pred=.;VEST3_score=.;VEST3_rankscore=.;MetaSVM_score=.;MetaSVM_rankscore=.;MetaSVM_pred=.;MetaLR_score=.;MetaLR_rankscore=.;MetaLR_pred=.;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=.;CADD_raw_rankscore=.;CADD_phred=.;DANN_score=.;DANN_rankscore=.;fathmm-MKL_coding_score=.;fathmm-MKL_coding_rankscore=.;fathmm-MKL_coding_pred=.;Eigen_coding_or_noncoding=.;Eigen-raw=.;Eigen-PC-raw=.;GenoCanyon_score=.;GenoCanyon_score_rankscore=.;integrated_fitCons_score=.;integrated_fitCons_score_rankscore=.;integrated_confidence_value=.;GERP++_RS=.;GERP++_RS_rankscore=.;phyloP100way_vertebrate=.;phyloP100way_vertebrate_rankscore=.;phyloP20way_mammalian=.;phyloP20way_mammalian_rankscore=.;phastCons100way_vertebrate=.;phastCons100way_vertebrate_rankscore=.;phastCons20way_mammalian=.;phastCons20way_mammalian_rankscore=.;SiPhy_29way_logOdds=.;SiPhy_29way_logOdds_rankscore=.;Interpro_domain=.;GTEx_V6_gene=.;GTEx_V6_tissue=.;esp6500siv2_all=.;esp6500siv2_aa=.;esp6500siv2_ea=.;ExAC_ALL=.;ExAC_AFR=.;ExAC_AMR=.;ExAC_EAS=.;ExAC_FIN=.;ExAC_NFE=.;ExAC_OTH=.;ExAC_SAS=.;ExAC_nontcga_ALL=.;ExAC_nontcga_AFR=.;ExAC_nontcga_AMR=.;ExAC_nontcga_EAS=.;ExAC_nontcga_FIN=.;ExAC_nontcga_NFE=.;ExAC_nontcga_OTH=.;ExAC_nontcga_SAS=.;ExAC_nonpsych_ALL=.;ExAC_nonpsych_AFR=.;ExAC_nonpsych_AMR=.;ExAC_nonpsych_EAS=.;ExAC_nonpsych_FIN=.;ExAC_nonpsych_NFE=.;ExAC_nonpsych_OTH=.;ExAC_nonpsych_SAS=.;1000g2015aug_all=.;1000g2015aug_afr=.;1000g2015aug_amr=.;1000g2015aug_eur=.;1000g2015aug_sas=.;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=.;avsnp150=.;CADD13_RawScore=0.015973;CADD13_PHRED=2.741;Eigen=-0.3239;REVEL=.;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0003;gnomAD_genome_AFR=0.0001;gnomAD_genome_AMR=0;gnomAD_genome_ASJ=0;gnomAD_genome_EAS=0.0007;gnomAD_genome_FIN=0.0009;gnomAD_genome_NFE=0.0002;gnomAD_genome_OTH=0.0011;gerp++gt2=.;cosmic70=.;InterVar_automated=.;PVS1=.;PS1=.;PS2=.;PS3=.;PS4=.;PM1=.;PM2=.;PM3=.;PM4=.;PM5=.;PM6=.;PP1=.;PP2=.;PP3=.;PP4=.;PP5=.;BA1=.;BS1=.;BS2=.;BS3=.;BS4=.;BP1=.;BP2=.;BP3=.;BP4=.;BP5=.;BP6=.;BP7=.;Kaviar_AF=.;Kaviar_AC=.;Kaviar_AN=.;ALLELE_END	GT:AD:DP:GQ:PL	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827	0/1:51,8:59:32:32,0,1388	0/0:47,0:47:99:0,114,1353	0/0:74,0:74:51:0,51,1827"""


assert create_dict_from_line(header, line) == solution

# Part 5 (10 points)
Write a function whose input is a filename for a vcf. The function reads the vcf file one variant at a time and transforms it into a dictionary using the create_dict_from_line function. It returns a list containing all the variant dictionaries. 

In [10]:
def read_vcf_file(filename):
    
    # Vars
    return_list = []
    index = 0 
    header = ""
    found_header = False
    read_file = open(filename , "r") 
    line_list = []
    # Loop Through Lines
    for i in read_file:  
        # Check if is header or after header
        if "#" not in i.replace("#","",1): 
            found_header = True
            
            if header == "" : 
                replace_header = ""
                replace_list = []
                header = i.split('\t')
                for i in range(0, len(header)):
                    replace_header = header[i].replace("#", "").replace("\n", "")
                    replace_list.append(replace_header)
                header = replace_list
            else:  
                to_return = create_dict_from_line(header, i) 
                return_list.append(to_return) 
            
        index +=1  
    return return_list 


#result = read_vcf_file('single_variant.vcf')
result = read_vcf_file('lab1_data.vcf')
#print(result)

In [11]:
expected_output = [{'ALT': 'G',
  'CHROM': '4',
  'FILTER': 'PASS',
  'ID': '.',
  'INFO': 'AC=1;AF=0.167;AN=6;BaseQRankSum=-2.542;ClippingRankSum=0;DP=180;ExcessHet=3.0103;FS=0;MLEAC=1;MLEAF=0.167;MQ=52.77;MQRankSum=-4.631;QD=0.39;ReadPosRankSum=1.45;SOR=0.758;VQSLOD=-8.209;culprit=MQ;ANNOVAR_DATE=2018-04-16;Func.refGene=intergenic;Gene.refGene=IL2\\x3bIL21;GeneDetail.refGene=dist\\x3d38536\\x3bdist\\x3d117597;ExonicFunc.refGene=.;AAChange.refGene=.;Func.ensGene=intergenic;Gene.ensGene=ENSG00000109471\\x3bENSG00000138684;GeneDetail.ensGene=dist\\x3d38306\\x3bdist\\x3d117597;ExonicFunc.ensGene=.;AAChange.ensGene=.;cytoBand=4q27;gwasCatalog=.;tfbsConsSites=.;wgRna=.;targetScanS=.;Gene_symbol=.;OXPHOS_Complex=.;Ensembl_Gene_ID=.;Ensembl_Protein_ID=.;Uniprot_Name=.;Uniprot_ID=.;NCBI_Gene_ID=.;NCBI_Protein_ID=.;Gene_pos=.;AA_pos=.;AA_sub=.;Codon_sub=.;dbSNP_ID=.;PhyloP_46V=.;PhastCons_46V=.;PhyloP_100V=.;PhastCons_100V=.;SiteVar=.;PolyPhen2_prediction=.;PolyPhen2_score=.;SIFT_prediction=.;SIFT_score=.;FatHmm_prediction=.;FatHmm_score=.;PROVEAN_prediction=.;PROVEAN_score=.;MutAss_prediction=.;MutAss_score=.;EFIN_Swiss_Prot_Score=.;EFIN_Swiss_Prot_Prediction=.;EFIN_HumDiv_Score=.;EFIN_HumDiv_Prediction=.;CADD_score=.;CADD_Phred_score=.;CADD_prediction=.;Carol_prediction=.;Carol_score=.;Condel_score=.;Condel_pred=.;COVEC_WMV=.;COVEC_WMV_prediction=.;PolyPhen2_score_transf=.;PolyPhen2_pred_transf=.;SIFT_score_transf=.;SIFT_pred_transf=.;MutAss_score_transf=.;MutAss_pred_transf=.;Perc_coevo_Sites=.;Mean_MI_score=.;COSMIC_ID=.;Tumor_site=.;Examined_samples=.;Mutation_frequency=.;US=.;Status=.;Associated_disease=.;Presence_in_TD=.;Class_predicted=.;Prob_N=.;Prob_P=.;SIFT_score=.;SIFT_converted_rankscore=.;SIFT_pred=.;Polyphen2_HDIV_score=.;Polyphen2_HDIV_rankscore=.;Polyphen2_HDIV_pred=.;Polyphen2_HVAR_score=.;Polyphen2_HVAR_rankscore=.;Polyphen2_HVAR_pred=.;LRT_score=.;LRT_converted_rankscore=.;LRT_pred=.;MutationTaster_score=.;MutationTaster_converted_rankscore=.;MutationTaster_pred=.;MutationAssessor_score=.;MutationAssessor_score_rankscore=.;MutationAssessor_pred=.;FATHMM_score=.;FATHMM_converted_rankscore=.;FATHMM_pred=.;PROVEAN_score=.;PROVEAN_converted_rankscore=.;PROVEAN_pred=.;VEST3_score=.;VEST3_rankscore=.;MetaSVM_score=.;MetaSVM_rankscore=.;MetaSVM_pred=.;MetaLR_score=.;MetaLR_rankscore=.;MetaLR_pred=.;M-CAP_score=.;M-CAP_rankscore=.;M-CAP_pred=.;CADD_raw=.;CADD_raw_rankscore=.;CADD_phred=.;DANN_score=.;DANN_rankscore=.;fathmm-MKL_coding_score=.;fathmm-MKL_coding_rankscore=.;fathmm-MKL_coding_pred=.;Eigen_coding_or_noncoding=.;Eigen-raw=.;Eigen-PC-raw=.;GenoCanyon_score=.;GenoCanyon_score_rankscore=.;integrated_fitCons_score=.;integrated_fitCons_score_rankscore=.;integrated_confidence_value=.;GERP++_RS=.;GERP++_RS_rankscore=.;phyloP100way_vertebrate=.;phyloP100way_vertebrate_rankscore=.;phyloP20way_mammalian=.;phyloP20way_mammalian_rankscore=.;phastCons100way_vertebrate=.;phastCons100way_vertebrate_rankscore=.;phastCons20way_mammalian=.;phastCons20way_mammalian_rankscore=.;SiPhy_29way_logOdds=.;SiPhy_29way_logOdds_rankscore=.;Interpro_domain=.;GTEx_V6_gene=.;GTEx_V6_tissue=.;esp6500siv2_all=.;esp6500siv2_aa=.;esp6500siv2_ea=.;ExAC_ALL=.;ExAC_AFR=.;ExAC_AMR=.;ExAC_EAS=.;ExAC_FIN=.;ExAC_NFE=.;ExAC_OTH=.;ExAC_SAS=.;ExAC_nontcga_ALL=.;ExAC_nontcga_AFR=.;ExAC_nontcga_AMR=.;ExAC_nontcga_EAS=.;ExAC_nontcga_FIN=.;ExAC_nontcga_NFE=.;ExAC_nontcga_OTH=.;ExAC_nontcga_SAS=.;ExAC_nonpsych_ALL=.;ExAC_nonpsych_AFR=.;ExAC_nonpsych_AMR=.;ExAC_nonpsych_EAS=.;ExAC_nonpsych_FIN=.;ExAC_nonpsych_NFE=.;ExAC_nonpsych_OTH=.;ExAC_nonpsych_SAS=.;1000g2015aug_all=.;1000g2015aug_afr=.;1000g2015aug_amr=.;1000g2015aug_eur=.;1000g2015aug_sas=.;CLNALLELEID=.;CLNDN=.;CLNDISDB=.;CLNREVSTAT=.;CLNSIG=.;dbscSNV_ADA_SCORE=.;dbscSNV_RF_SCORE=.;snp138NonFlagged=.;avsnp150=.;CADD13_RawScore=0.015973;CADD13_PHRED=2.741;Eigen=-0.3239;REVEL=.;MCAP=.;Interpro_domain=.;ICGC_Id=.;ICGC_Occurrence=.;gnomAD_genome_ALL=0.0003;gnomAD_genome_AFR=0.0001;gnomAD_genome_AMR=0;gnomAD_genome_ASJ=0;gnomAD_genome_EAS=0.0007;gnomAD_genome_FIN=0.0009;gnomAD_genome_NFE=0.0002;gnomAD_genome_OTH=0.0011;gerp++gt2=.;cosmic70=.;InterVar_automated=.;PVS1=.;PS1=.;PS2=.;PS3=.;PS4=.;PM1=.;PM2=.;PM3=.;PM4=.;PM5=.;PM6=.;PP1=.;PP2=.;PP3=.;PP4=.;PP5=.;BA1=.;BS1=.;BS2=.;BS3=.;BS4=.;BP1=.;BP2=.;BP3=.;BP4=.;BP5=.;BP6=.;BP7=.;Kaviar_AF=.;Kaviar_AC=.;Kaviar_AN=.;ALLELE_END',
  'POS': '123416186',
  'QUAL': '23.25',
  'REF': 'A',
  'SAMPLE': {'XG102': {'AD': '51,8',
                       'DP': '59',
                       'GQ': '32',
                       'GT': '0/1',
                       'PL': '32,0,1388'},
             'XG103': {'AD': '47,0',
                       'DP': '47',
                       'GQ': '99',
                       'GT': '0/0',
                       'PL': '0,114,1353'},
             'XG104': {'AD': '74,0',
                       'DP': '74',
                       'GQ': '51',
                       'GT': '0/0',
                       'PL': '0,51,1827'},
             'XG202': {'AD': '51,8',
                       'DP': '59',
                       'GQ': '32',
                       'GT': '0/1',
                       'PL': '32,0,1388'},
             'XG203': {'AD': '47,0',
                       'DP': '47',
                       'GQ': '99',
                       'GT': '0/0',
                       'PL': '0,114,1353'},
             'XG204': {'AD': '74,0',
                       'DP': '74',
                       'GQ': '51',
                       'GT': '0/0',
                       'PL': '0,51,1827'},
             'XG302': {'AD': '51,8',
                       'DP': '59',
                       'GQ': '32',
                       'GT': '0/1',
                       'PL': '32,0,1388'},
             'XG303': {'AD': '47,0',
                       'DP': '47',
                       'GQ': '99',
                       'GT': '0/0',
                       'PL': '0,114,1353'},
             'XG304': {'AD': '74,0',
                       'DP': '74',
                       'GQ': '51',
                       'GT': '0/0',
                       'PL': '0,51,1827'},
             'XG402': {'AD': '51,8',
                       'DP': '59',
                       'GQ': '32',
                       'GT': '0/1',
                       'PL': '32,0,1388'},
             'XG403': {'AD': '47,0',
                       'DP': '47',
                       'GQ': '99',
                       'GT': '0/0',
                       'PL': '0,114,1353'},
             'XG404': {'AD': '74,0',
                       'DP': '74',
                       'GQ': '51',
                       'GT': '0/0',
                       'PL': '0,51,1827'}}}]

result = read_vcf_file('single_variant.vcf')
assert expected_output == result

# Part 6 (5 points)
Write a function that extracts the info field from the data dictionary that was created in the previous part. The function should return all the info field dictionaries as list. 


In [12]:
def extract_info_field(data):
    return_list = []
    for i in data:
            #print(i["INFO"])
            return_list.append(i["INFO"])
    #print(return_list)
    return return_list
#Eexpected_results = open('info_field.vcf', 'r').readline().strip()
#print(expected_results)
#info_field = extract_info_field(result)
#print(info_field)

In [13]:
expected_results = open('info_field.vcf', 'r').readline().strip()
info_field = extract_info_field(result)

assert expected_results == info_field[0]


# Part 7 (10 points)
You now need to figure out that data types for each of the info fields. 
Begin by writing a function that first takes the info fields and turns them into a dictionary. Make sure to skip any fields that do not have a value or are missing a value. Also replace \\x3b with a comma and \\x3d with an equal sign.  

In [14]:
def create_dictionary_of_info_field_values(data): 
    return_list = {} 
    
    #loop through datas
    for i in range(0, len(data)): 
        data_list = data[i].split(";") 
        
        # loop through individual key-values
        for i in data_list:   
            new_i = ""
            new_i = i.replace("\\x3b",",").replace("\\x3d","=")
            
            split_i_key_value = [] 
            #if have more than 1 equal signs
            if "=" in new_i:
                split_i_key_value = new_i.split("=",1)  
                #print(split_i_key_value)
                
                try: 
                    # CHECK IF LIST EXIST? APPEND:CREATE
                       
                    #tester = type(return_list[split_i_key_value[0]] )
                    #print(tester)
                    #print(return_list[split_i_key_value[0]])
                    #if split_i_key_value[1] == ".": 
                    #    split_i_key_value[1] = []
                    #     
                    #else :
                    #    # toss in list
                    #    split_i_key_value[1] = [split_i_key_value[1]]
                    # 
                    if split_i_key_value[1] != ".":
                        #split_i_key_value[1] = [split_i_key_value[1]]
                        #return_list[split_i_key_value[0]].append = split_i_key_value[1]
                        #print(split_key_value[1])
                        if split_i_key_value[0] in return_list:
#                             print("locate error1")
                            return_list[split_i_key_value[0]].append(split_i_key_value[1]) 
                        else:
#                             print("locate error2")
#                             print("split_i_key_value[0]" , split_i_key_value[0])
#                             print("split_i_key_value[1]", split_i_key_value[1])
                            return_list[split_i_key_value[0]] = [split_i_key_value[1]]
                    else:
#                         print("locate error 4")
                        if split_i_key_value[0] not in return_list:
                            return_list[split_i_key_value[0]] = [] 
                except :
#                     print("has error")
                    pass
                
    #print("Q7", return_list)
    return return_list


In [15]:

info_field = extract_info_field(result)
expected_results = {'AC': ['1'], 'AF': ['0.167'], 'AN': ['6'], 'BaseQRankSum': ['-2.542'], 'ClippingRankSum': ['0'], 'DP': ['180'], 'ExcessHet': ['3.0103'], 'FS': ['0'], 'MLEAC': ['1'], 'MLEAF': ['0.167'], 'MQ': ['52.77'], 'MQRankSum': ['-4.631'], 'QD': ['0.39'], 'ReadPosRankSum': ['1.45'], 'SOR': ['0.758'], 'VQSLOD': ['-8.209'], 'culprit': ['MQ'], 'ANNOVAR_DATE': ['2018-04-16'], 'Func.refGene': ['intergenic'], 'Gene.refGene': ['IL2,IL21'], 'GeneDetail.refGene': ['dist=38536,dist=117597'], 'ExonicFunc.refGene': [], 'AAChange.refGene': [], 'Func.ensGene': ['intergenic'], 'Gene.ensGene': ['ENSG00000109471,ENSG00000138684'], 'GeneDetail.ensGene': ['dist=38306,dist=117597'], 'ExonicFunc.ensGene': [], 'AAChange.ensGene': [], 'cytoBand': ['4q27'], 'gwasCatalog': [], 'tfbsConsSites': [], 'wgRna': [], 'targetScanS': [], 'Gene_symbol': [], 'OXPHOS_Complex': [], 'Ensembl_Gene_ID': [], 'Ensembl_Protein_ID': [], 'Uniprot_Name': [], 'Uniprot_ID': [], 'NCBI_Gene_ID': [], 'NCBI_Protein_ID': [], 'Gene_pos': [], 'AA_pos': [], 'AA_sub': [], 'Codon_sub': [], 'dbSNP_ID': [], 'PhyloP_46V': [], 'PhastCons_46V': [], 'PhyloP_100V': [], 'PhastCons_100V': [], 'SiteVar': [], 'PolyPhen2_prediction': [], 'PolyPhen2_score': [], 'SIFT_prediction': [], 'SIFT_score': [], 'FatHmm_prediction': [], 'FatHmm_score': [], 'PROVEAN_prediction': [], 'PROVEAN_score': [], 'MutAss_prediction': [], 'MutAss_score': [], 'EFIN_Swiss_Prot_Score': [], 'EFIN_Swiss_Prot_Prediction': [], 'EFIN_HumDiv_Score': [], 'EFIN_HumDiv_Prediction': [], 'CADD_score': [], 'CADD_Phred_score': [], 'CADD_prediction': [], 'Carol_prediction': [], 'Carol_score': [], 'Condel_score': [], 'Condel_pred': [], 'COVEC_WMV': [], 'COVEC_WMV_prediction': [], 'PolyPhen2_score_transf': [], 'PolyPhen2_pred_transf': [], 'SIFT_score_transf': [], 'SIFT_pred_transf': [], 'MutAss_score_transf': [], 'MutAss_pred_transf': [], 'Perc_coevo_Sites': [], 'Mean_MI_score': [], 'COSMIC_ID': [], 'Tumor_site': [], 'Examined_samples': [], 'Mutation_frequency': [], 'US': [], 'Status': [], 'Associated_disease': [], 'Presence_in_TD': [], 'Class_predicted': [], 'Prob_N': [], 'Prob_P': [], 'SIFT_converted_rankscore': [], 'SIFT_pred': [], 'Polyphen2_HDIV_score': [], 'Polyphen2_HDIV_rankscore': [], 'Polyphen2_HDIV_pred': [], 'Polyphen2_HVAR_score': [], 'Polyphen2_HVAR_rankscore': [], 'Polyphen2_HVAR_pred': [], 'LRT_score': [], 'LRT_converted_rankscore': [], 'LRT_pred': [], 'MutationTaster_score': [], 'MutationTaster_converted_rankscore': [], 'MutationTaster_pred': [], 'MutationAssessor_score': [], 'MutationAssessor_score_rankscore': [], 'MutationAssessor_pred': [], 'FATHMM_score': [], 'FATHMM_converted_rankscore': [], 'FATHMM_pred': [], 'PROVEAN_converted_rankscore': [], 'PROVEAN_pred': [], 'VEST3_score': [], 'VEST3_rankscore': [], 'MetaSVM_score': [], 'MetaSVM_rankscore': [], 'MetaSVM_pred': [], 'MetaLR_score': [], 'MetaLR_rankscore': [], 'MetaLR_pred': [], 'M-CAP_score': [], 'M-CAP_rankscore': [], 'M-CAP_pred': [], 'CADD_raw': [], 'CADD_raw_rankscore': [], 'CADD_phred': [], 'DANN_score': [], 'DANN_rankscore': [], 'fathmm-MKL_coding_score': [], 'fathmm-MKL_coding_rankscore': [], 'fathmm-MKL_coding_pred': [], 'Eigen_coding_or_noncoding': [], 'Eigen-raw': [], 'Eigen-PC-raw': [], 'GenoCanyon_score': [], 'GenoCanyon_score_rankscore': [], 'integrated_fitCons_score': [], 'integrated_fitCons_score_rankscore': [], 'integrated_confidence_value': [], 'GERP++_RS': [], 'GERP++_RS_rankscore': [], 'phyloP100way_vertebrate': [], 'phyloP100way_vertebrate_rankscore': [], 'phyloP20way_mammalian': [], 'phyloP20way_mammalian_rankscore': [], 'phastCons100way_vertebrate': [], 'phastCons100way_vertebrate_rankscore': [], 'phastCons20way_mammalian': [], 'phastCons20way_mammalian_rankscore': [], 'SiPhy_29way_logOdds': [], 'SiPhy_29way_logOdds_rankscore': [], 'Interpro_domain': [], 'GTEx_V6_gene': [], 'GTEx_V6_tissue': [], 'esp6500siv2_all': [], 'esp6500siv2_aa': [], 'esp6500siv2_ea': [], 'ExAC_ALL': [], 'ExAC_AFR': [], 'ExAC_AMR': [], 'ExAC_EAS': [], 'ExAC_FIN': [], 'ExAC_NFE': [], 'ExAC_OTH': [], 'ExAC_SAS': [], 'ExAC_nontcga_ALL': [], 'ExAC_nontcga_AFR': [], 'ExAC_nontcga_AMR': [], 'ExAC_nontcga_EAS': [], 'ExAC_nontcga_FIN': [], 'ExAC_nontcga_NFE': [], 'ExAC_nontcga_OTH': [], 'ExAC_nontcga_SAS': [], 'ExAC_nonpsych_ALL': [], 'ExAC_nonpsych_AFR': [], 'ExAC_nonpsych_AMR': [], 'ExAC_nonpsych_EAS': [], 'ExAC_nonpsych_FIN': [], 'ExAC_nonpsych_NFE': [], 'ExAC_nonpsych_OTH': [], 'ExAC_nonpsych_SAS': [], '1000g2015aug_all': [], '1000g2015aug_afr': [], '1000g2015aug_amr': [], '1000g2015aug_eur': [], '1000g2015aug_sas': [], 'CLNALLELEID': [], 'CLNDN': [], 'CLNDISDB': [], 'CLNREVSTAT': [], 'CLNSIG': [], 'dbscSNV_ADA_SCORE': [], 'dbscSNV_RF_SCORE': [], 'snp138NonFlagged': [], 'avsnp150': [], 'CADD13_RawScore': ['0.015973'], 'CADD13_PHRED': ['2.741'], 'Eigen': ['-0.3239'], 'REVEL': [], 'MCAP': [], 'ICGC_Id': [], 'ICGC_Occurrence': [], 'gnomAD_genome_ALL': ['0.0003'], 'gnomAD_genome_AFR': ['0.0001'], 'gnomAD_genome_AMR': ['0'], 'gnomAD_genome_ASJ': ['0'], 'gnomAD_genome_EAS': ['0.0007'], 'gnomAD_genome_FIN': ['0.0009'], 'gnomAD_genome_NFE': ['0.0002'], 'gnomAD_genome_OTH': ['0.0011'], 'gerp++gt2': [], 'cosmic70': [], 'InterVar_automated': [], 'PVS1': [], 'PS1': [], 'PS2': [], 'PS3': [], 'PS4': [], 'PM1': [], 'PM2': [], 'PM3': [], 'PM4': [], 'PM5': [], 'PM6': [], 'PP1': [], 'PP2': [], 'PP3': [], 'PP4': [], 'PP5': [], 'BA1': [], 'BS1': [], 'BS2': [], 'BS3': [], 'BS4': [], 'BP1': [], 'BP2': [], 'BP3': [], 'BP4': [], 'BP5': [], 'BP6': [], 'BP7': [], 'Kaviar_AF': [], 'Kaviar_AC': [], 'Kaviar_AN': []}
results = create_dictionary_of_info_field_values(info_field)
assert expected_results == results



# Part 8 (5 points)
Write a function whose input is the output from `create_dictionary_of_info_field_values` and uses the previously written function `determine_data_type_of_list` to determine the data type of each of the info fields. The output is a dictionary whose keys are the name of the info fields and values are the data type.  

In [16]:
data = {'key1': ['1', '2', '3'], 'key2':['1.1', '2.2', '3.3'], 'key3': ['1.1', '2', '3.3'], 'key4': ['1.1', '234String', '3.3']}

def determine_data_type_of_info_fields(data): 
    return_dict = {}
    for i in data:
        #print(data[i])
        return_dict[i] = determine_data_type_of_list(data[i])
#     print(return_dict)
    return return_dict
temp_data = { "ALT": "G","CHROM": "4","1000g2015aug_sas":"0.432","FILTER": "PASS","ID": ".","QUAL" : '23.25' , "TESTER": ['2.2' , '3.3', '4.4'] }
#print(determine_data_type_of_info_fields(temp_data))
#determine_data_type_of_info_fields(data)
#temp_data = { "ALT": "G","CHROM": "4","FILTER": "PASS","ID": ".","QUAL" : '23.25' , "TESTER": ['2.2' , '3.3', '4.4'] }
#print(determine_data_type_of_info_fields(temp_data))

In [17]:
data = {'key1': ['1', '2', '3'], 'key2':['1.1', '2.2', '3.3'], 'key3': ['1.1', '2', '3.3'], 'key4': ['1.1', '234String', '3.3']}
expected_results = {'key1': int, 'key2': float, 'key3': float, 'key4': str}

assert determine_data_type_of_info_fields(data) == expected_results


# Part 9  (No points)
Write a function whose first input is the data from  `read_vcf_file` and the second input is the output from `determine_data_type_of_info_fields`. The function converts the info field into a dictionary and uses the data types that you determined to cast each field into the correct data type. Make sure to convert the `POS` to int and `QUAL` to float. Replace any `\\x3b` with a comma and any `\\x3d` with an equal sign. The output will look something like this (I have removed most of the fields):

The output will look something like this 
```
{
        "ALT": "G",
        "CHROM": "4",
        "FILTER": "PASS",
        "ID": ".",
        "INFO": {
           
            "Gene.ensGene": "ENSG00000109471,ENSG00000138684",
            "Gene.refGene": "IL2,IL21",
            "GeneDetail.ensGene": "dist=38306,dist=117597",
            "GeneDetail.refGene": "dist=38536,dist=117597"
        },
        "POS": 123416186,
        "QUAL" :23.25,
        "REF": "A",
        "SAMPLE": {
            "XG102": {
                "AD": "51,8",
                "DP": "59",
                "GQ": "32",
                "GT": "0/1",
                "PL": "32,0,1388"
            }
    }
```

In [18]:
# #filename = 'test_answer_single.vcf' 
# filename = 'test_the_answer.vcf' 

# data = read_vcf_file(filename) # read vcf file  
# info_field_data = extract_info_field(data) # extract all the info fields 
# print("info_field_data",info_field_data)

# info_field_list = create_dictionary_of_info_field_values(info_field_data) # create dictionary from info fields

# print("info_field_list" , info_field_list)
# info_field_data_type = determine_data_type_of_info_fields(info_field_list) # Determine data type of each info field

# print()
# print("DATA" , data[0])
# print()
# print("DATA" , data[1])
# print()

# print()
# print("info_field" , info_field_data_type) 
# print()

# data = format_data(data, info_field_data_type) # format the data variable -- from data = read_vcf_file(filename)

# print("new data", data)


In [19]:
#input list dict 
# return list tdict

# data is output from read_vcf
# info_field_data_type is output from determine_data_type_of_info_fields
# 1) The function converts the info field  ## 1st vairable
# 2)into a dictionary and 
# uses the data types that you determined 
# 3) to cast each field into the correct data type
def format_data(data, info_field_data_type):
    #print(info_field_data_type)
    # INFO section
    
    #return list returns the converted value of the datas
    return_list = []
    
    
    # Data is All data input
    for i in data:
        
        return_info = {}
        #print(i["INFO"])
        #header_i is key for listdatas
        for header_i in i:
             
            if header_i == "INFO": 
                info_dict = {}
                # TODO problem is here
                # 不要輸入不需要的欄位到return_info裡面
                temp_i = i["INFO"].split(";") 
                
                #print("temp_i" , temp_i)
                # Checked

                for q in temp_i:
                    #q is each key-value in info 
                    temp_q = q.split("=")  
                    #if temp_q[0] == "1000g2015aug_all":
                    #    print(temp_q[1])
                    #    print(info_field_data_type[temp_q[0]])
                    
                    
                    try:
                        # deleter empty values in tnfo
                        if temp_q[1] == "." or len(temp_q) == 1:
                            #if temp_q[0] == "1000g2015aug_all":
                            #    print("Entered del")
                            del temp_i
                        else:
                            #if temp_q[0] == "1000g2015aug_all":
                            #    print("Entered normal")
                            # replace values in info
                            temp_q[1] = temp_q[1].replace("\\x3b",",").replace("\\x3d","=")
                            
                            # transform data type in info
                            #print("temp_q[0]" , temp_q[0], ". temp_q[1]", temp_q[1])
                            #print("info_field_data_type , ", info_field_data_type[temp_q[0]])
                            if info_field_data_type[temp_q[0]] == int:
                                 
                                temp_q[1] = int(temp_q[1])
                                #print("new i int" , temp_q[1])
                                info_dict[temp_q[0]] = temp_q[1]
                            if info_field_data_type[temp_q[0]] == float:
                                #if temp_q[0] == "1000g2015aug_all":
                                #    print("Entered normal float")
                                temp_q[1] = float(temp_q[1])
                                #print("new i float" , temp_q[1])
                                
                                info_dict[temp_q[0]] = temp_q[1]
                                #if temp_q[0] == "1000g2015aug_all":
                                #    print("return_info[temp_q[0]]" , return_info[temp_q[0]])
                            if info_field_data_type[temp_q[0]] == str:
                                temp_q[1] = str(temp_q[1])
                                #print("new i int" , temp_q[1])
                                info_dict[temp_q[0]] = temp_q[1]
                    except:
                        pass
                i[header_i] = return_info
                return_info["INFO"] = info_dict
            elif header_i == "POS":
                return_info[header_i] = int(i[header_i])
                #i[header_i] = int(i[header_i])
            elif header_i == "QUAL":
                return_info[header_i] = float(i[header_i])
                i[header_i] = float(i[header_i]) 
            else: 
                return_info[header_i] = i[header_i]
        return_list.append(return_info)
    #return(data)
    #print("return_info ", return_list)
    return(return_list)
        #print(return_info)

    
#### ==== VERIFY ====

##input 1
#result = read_vcf_file('single_variant.vcf')
#
##input 2
#variant_line = determine_data_type_of_info_fields(result[0])
#
#
## print(variant_line)
## print(result)

#format_data(result,variant_line )

# Part 10 (No Points)
Write a function whose inputs are a Python dictionary and filename. The function will saves the dictionary as a json file using the filename given. Use the json library. Use these options to correctly format your JSON -- `sort_keys=True, indent=4, separators=(',', ': ')`. 
Use this function to save your parsed data as a json file. 

USE THE FOLLOWING FILENAME TO SAVE YOUR FILE: `lab1.json`

In [20]:
import json
def save_data_as_json(data, filename): 
    
    with open(filename, 'w') as outfile:
        json.dump(data, outfile, sort_keys=True, indent=4, separators=(',', ': '))

        
#### === VERIFY ===
##input 1 is dictionary
#input1 = {'key1': ['1', '2', '3'], 'key2':['1.1', '2.2', '3.3'], 'key3': ['1.1', '2', '3.3'], 'key4': ['1.1', '234String', '3.3']}
##input 2 is filename
#input2 = "lab1.json"
#save_data_as_json(input1, input2)

# Part 11 (10 points)
Write a function whose input is a filename for a json file. The function should use the filename to read the JSON file in which you saved your final parsed data. 

In [21]:
def load_data_from_json(filename):
    data = {}
    with open(filename,'r') as json_file:
        data = json.load(json_file) 
    return data


In [22]:
test_save_data = {'key1': ['1', '2', '3'], 'key2':['1.1', '2.2', '3.3'], 'key3': ['1.1', '2', '3.3'], 'key4': ['1.1', '234String', '3.3']}
save_data_as_json(test_save_data, 'test_save.json')
results = load_data_from_json('test_save.json')

assert results == test_save_data


# Part 12 (30 points)
Write a function whose inputs are `CHROM`, `REF`, `ALT`, `POS`, and `filename`.  Using these inputs, the function should load a JSON file using the given filename and return a list of variants that match 
the given CHROM, REF, ALT, and POS. 

In [23]:
def find_variant(CHROM, REF, ALT, POS, filename):
    found_count = 0
    return_list = []
    #print("data" , data)
    #print(" :D ") 
    index_i = 0 
    for i in data: 
        #print("Q12" , i)
        if i["CHROM"] == CHROM and i["REF"] == REF and i["ALT"] == ALT and i["POS"] == POS:
            #print(i)
            #print(i)
          #  print("Found at index :" ,index_i)
            found_count +=1
            return_list.append(i)
        index_i += 1
        #print(index_i)
        
        #print("\n")
    #print("Found count ", found_count)
    #print("Return_list " , return_list)
    return return_list

# filename = 'single_variant.vcf' 
# #filename = 'lab1_data.vcf' 
# data = read_vcf_file(filename) # read vcf file 
# info_field_data = extract_info_field(data) # extract all the info fields
# info_field_list = create_dictionary_of_info_field_values(info_field_data) # create dictionary from info fields
# info_field_data_type = determine_data_type_of_info_fields(info_field_list) # Determine data type of each info field
# #print(data[0],"\n\n\n")
# #print(info_field_data_type)
# #print(info_field_data_type)
# data = format_data(data, info_field_data_type) # format the data variable -- from data = read_vcf_file(filename)
# #print(data[0])


# save_data_as_json(data, 'lab1.json') # save the formatted data
# data_loaded = load_data_from_json('lab1.json') # load saved data 

# # Now I will fetch three variants using your find_variant function and check that they match my results



In [24]:
## === My Own Test Files ===


# #filename = 'test_answer_single.vcf' 
# filename = 'test_the_answer.vcf' 

# data = read_vcf_file(filename) # read vcf file  
# info_field_data = extract_info_field(data) # extract all the info fields 
# info_field_list = create_dictionary_of_info_field_values(info_field_data) # create dictionary from info fields

# info_field_data_type = determine_data_type_of_info_fields(info_field_list) # Determine data type of each info field

# print()
# print("DATA" , data[0])
# print()
# print("DATA" , data[1])
# print()

# print()
# print("info_field" , info_field_data_type) 
# print()

# data = format_data(data, info_field_data_type) # format the data variable -- from data = read_vcf_file(filename)

# print("new data", data)

# # irrelevant
# save_data_as_json(data, 'test_the_answer.json') # save the formatted data
# data_loaded = load_data_from_json('test_the_answer.json') # load saved data 

# # Now I will fetch three variants using your find_variant function and check that they match my results


# result_answer = find_variant("13", "T", "G", 56292303, 'test_the_answer.json')
# save_data_as_json(result_answer, 'result_test_answer.json') # save the formatted data
 
    
# # TODO DOESNT PASS WHEN JSON IDENTICAL

# expected_result1 = [{u'INFO': {u'1000g2015aug_eur': 1.0, u'BaseQRankSum': 1.77, u'Gene.refGene': u'MIR5007,PRR20D', u'cytoBand': u'13q21.1', u'gnomAD_genome_FIN': 0.9951, u'1000g2015aug_amr': 0.9841, u'gnomAD_genome_AFR': 0.8475, u'FS': 0.0, u'gnomAD_genome_NFE': 0.9987, u'MQRankSum': 0.0, u'snp138NonFlagged': u'rs4421887', u'gnomAD_genome_ALL': 0.9546, u'DP': 176, u'ExcessHet': 3.0103, u'gnomAD_genome_OTH': 0.9887, u'MLEAF': 1.0, u'culprit': u'MQ', u'1000g2015aug_afr': 0.8313, u'SOR': 0.257, u'VQSLOD': 22.53, u'MQ': 60.0, u'ClippingRankSum': 0, u'Kaviar_AF': 0.0001153, u'AC': 6, u'Kaviar_AC': 3, u'AF': 1.0, u'Kaviar_AN': 26028, u'Func.refGene': u'intergenic', u'AN': 6, u'CADD13_PHRED': 3.712, u'1000g2015aug_all': 0.952676, u'CADD13_RawScore': 0.109684, u'1000g2015aug_sas': 0.9969, u'GeneDetail.ensGene': u'dist=543620,dist=281031', u'ANNOVAR_DATE': u'2018-04-16', u'Func.ensGene': u'intergenic', u'ReadPosRankSum': -0.338, u'gnomAD_genome_ASJ': 0.9834, u'gnomAD_genome_AMR': 0.9843, u'Eigen': -0.7945, u'avsnp150': u'rs4421887', u'MLEAC': 6, u'gnomAD_genome_EAS': 1.0, u'GeneDetail.refGene': u'dist=543620,dist=1422749', u'QD': 28.95, u'Gene.ensGene': u'ENSG00000264387,ENSG00000228611'}, u'SAMPLE': {u'XG404': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG402': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG403': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG303': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG302': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG304': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG202': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG203': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG204': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG103': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG102': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG104': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}}, u'REF': u'T', u'POS': 56292303, u'FILTER': u'PASS', u'QUAL': 5037.69, u'ALT': u'G', u'CHROM': u'13', u'ID': u'rs4421887'}]
# save_data_as_json(expected_result1, 'expected.json') # save the formatted data



# assert expected_result1==result_answer


In [25]:
## === My Own Test Files ===

#my_answer = result_answer[0]
#expected_answer = expected_result1[0]
#my_answer_info = my_answer["INFO"]
#my_answer_sample = my_answer["SAMPLE"]
#expected_answer_info = expected_answer["INFO"]
#expected_answer_sample = expected_answer["SAMPLE"]

#for i in expected_result1:
#    #print(i)
#    for q in i:
#        print(q)
#        if i[q] == my_answer[q]:
#            print(q, " is correct")
#        
##print("result_answer " ,result_answer[0])\
#
##compare smaple
#for i in expected_answer_sample:
#    if expected_answer_sample[i] != my_answer_sample[i]:
#        print("INCORRECT at " ,i, "expected_answer_sample[i]" , expected_answer_sample[i] )
#    else:
#        print("correct at " ,i, "expected_answer_sample[i]" , expected_answer_sample[i] )
#for i in my_answer:
#    if my_answer[i] != my_answer[i]:
#        print("INCORRECT at " ,i, "expected_answer_sample[i]" , my_answer[i] )
#    else:
#        print("correct at " ,i, "expected_answer_sample[i]" , my_answer[i] )
##for i in expected_answer_info:

In [29]:





filename = 'lab1_data.vcf' 
data = read_vcf_file(filename) # read vcf file 
info_field_data = extract_info_field(data) # extract all the info fields
info_field_list = create_dictionary_of_info_field_values(info_field_data) # create dictionary from info fields
info_field_data_type = determine_data_type_of_info_fields(info_field_list) # Determine data type of each info field
data = format_data(data, info_field_data_type) # format the data variable -- from data = read_vcf_file(filename)
save_data_as_json(data, 'lab1.json') # save the formatted data
data_loaded = load_data_from_json('lab1.json') # load saved data 

# Now I will fetch three variants using your find_variant function and check that they match my results



In [30]:

result1 = find_variant("13", "T", "G", 56292303, 'lab1.json')


expected_result1 = [{u'INFO': {u'1000g2015aug_eur': 1.0, u'BaseQRankSum': 1.77, u'Gene.refGene': u'MIR5007,PRR20D', u'cytoBand': u'13q21.1', u'gnomAD_genome_FIN': 0.9951, u'1000g2015aug_amr': 0.9841, u'gnomAD_genome_AFR': 0.8475, u'FS': 0.0, u'gnomAD_genome_NFE': 0.9987, u'MQRankSum': 0.0, u'snp138NonFlagged': u'rs4421887', u'gnomAD_genome_ALL': 0.9546, u'DP': 176, u'ExcessHet': 3.0103, u'gnomAD_genome_OTH': 0.9887, u'MLEAF': 1.0, u'culprit': u'MQ', u'1000g2015aug_afr': 0.8313, u'SOR': 0.257, u'VQSLOD': 22.53, u'MQ': 60.0, u'ClippingRankSum': 0, u'Kaviar_AF': 0.0001153, u'AC': 6, u'Kaviar_AC': 3, u'AF': 1.0, u'Kaviar_AN': 26028, u'Func.refGene': u'intergenic', u'AN': 6, u'CADD13_PHRED': 3.712, u'1000g2015aug_all': 0.952676, u'CADD13_RawScore': 0.109684, u'1000g2015aug_sas': 0.9969, u'GeneDetail.ensGene': u'dist=543620,dist=281031', u'ANNOVAR_DATE': u'2018-04-16', u'Func.ensGene': u'intergenic', u'ReadPosRankSum': -0.338, u'gnomAD_genome_ASJ': 0.9834, u'gnomAD_genome_AMR': 0.9843, u'Eigen': -0.7945, u'avsnp150': u'rs4421887', u'MLEAC': 6, u'gnomAD_genome_EAS': 1.0, u'GeneDetail.refGene': u'dist=543620,dist=1422749', u'QD': 28.95, u'Gene.ensGene': u'ENSG00000264387,ENSG00000228611'}, u'SAMPLE': {u'XG404': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG402': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG403': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG303': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG302': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG304': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG202': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG203': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG204': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG103': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG102': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG104': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}}, u'REF': u'T', u'POS': 56292303, u'FILTER': u'PASS', u'QUAL': 5037.69, u'ALT': u'G', u'CHROM': u'13', u'ID': u'rs4421887'}]


assert expected_result1==result1

In [28]:


## === My Own Test Files ===


# filename = 'lab1_data.vcf' 
# data = read_vcf_file(filename) # read vcf file 
# info_field_data = extract_info_field(data) # extract all the info fields
# info_field_list = create_dictionary_of_info_field_values(info_field_data) # create dictionary from info fields
# info_field_data_type = determine_data_type_of_info_fields(info_field_list) # Determine data type of each info field
# data = format_data(data, info_field_data_type) # format the data variable -- from data = read_vcf_file(filename)
# save_data_as_json(data, 'lab1.json') # save the formatted data
# data_loaded = load_data_from_json('lab1.json') # load saved data 

# # Now I will fetch three variants using your find_variant function and check that they match my results
 
# result1 = find_variant("13", "T", "G", 56292303, 'lab1.json') 

# save_data_as_json(result1, 'result1.json') # save the formatted data



# expected_result1 = [{u'INFO': {u'1000g2015aug_eur': 1.0, u'BaseQRankSum': 1.77, u'Gene.refGene': u'MIR5007,PRR20D', u'cytoBand': u'13q21.1', u'gnomAD_genome_FIN': 0.9951, u'1000g2015aug_amr': 0.9841, u'gnomAD_genome_AFR': 0.8475, u'FS': 0.0, u'gnomAD_genome_NFE': 0.9987, u'MQRankSum': 0.0, u'snp138NonFlagged': u'rs4421887', u'gnomAD_genome_ALL': 0.9546, u'DP': 176, u'ExcessHet': 3.0103, u'gnomAD_genome_OTH': 0.9887, u'MLEAF': 1.0, u'culprit': u'MQ', u'1000g2015aug_afr': 0.8313, u'SOR': 0.257, u'VQSLOD': 22.53, u'MQ': 60.0, u'ClippingRankSum': 0, u'Kaviar_AF': 0.0001153, u'AC': 6, u'Kaviar_AC': 3, u'AF': 1.0, u'Kaviar_AN': 26028, u'Func.refGene': u'intergenic', u'AN': 6, u'CADD13_PHRED': 3.712, u'1000g2015aug_all': 0.952676, u'CADD13_RawScore': 0.109684, u'1000g2015aug_sas': 0.9969, u'GeneDetail.ensGene': u'dist=543620,dist=281031', u'ANNOVAR_DATE': u'2018-04-16', u'Func.ensGene': u'intergenic', u'ReadPosRankSum': -0.338, u'gnomAD_genome_ASJ': 0.9834, u'gnomAD_genome_AMR': 0.9843, u'Eigen': -0.7945, u'avsnp150': u'rs4421887', u'MLEAC': 6, u'gnomAD_genome_EAS': 1.0, u'GeneDetail.refGene': u'dist=543620,dist=1422749', u'QD': 28.95, u'Gene.ensGene': u'ENSG00000264387,ENSG00000228611'}, u'SAMPLE': {u'XG404': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG402': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG403': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG303': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG302': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG304': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG202': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG203': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG204': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}, u'XG103': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,41', u'DP': u'41', u'PL': u'1218,122,0'}, u'XG102': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'0,62', u'DP': u'62', u'PL': u'1796,185,0'}, u'XG104': {u'GT': u'1/1', u'GQ': u'99', u'AD': u'1,70', u'DP': u'71', u'PL': u'2037,202,0'}}, u'REF': u'T', u'POS': 56292303, u'FILTER': u'PASS', u'QUAL': 5037.69, u'ALT': u'G', u'CHROM': u'13', u'ID': u'rs4421887'}]
# #print("Expected answers :", expected_result1)
# save_data_as_json(expected_result1, 'expected.json') # save the formatted data


# assert expected_result1==result1